In [21]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd 

dataset = load_dataset("sms_spam")
dataset = dataset.rename_column("sms", "text")
dataset = dataset['train']

processed_text = dataset['text']
display(type(processed_text))
processed_text = [s.replace('\n', '') for s in processed_text]

dataset = dataset.remove_columns(["text"])
display(dataset)
dataset = dataset.add_column("text", processed_text)
display(dataset)
dataset = dataset.shuffle(seed=42).select([i for i in list(range(5500))])


train_ds = Dataset.from_dict(dataset[0:4500])
validation_ds = Dataset.from_dict(dataset[4500:5000])
test_ds = Dataset.from_dict(dataset[5000:5500])


display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Reusing dataset sms_spam (C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.29it/s]


list

Dataset({
    features: ['label'],
    num_rows: 5574
})

Dataset({
    features: ['label', 'text'],
    num_rows: 5574
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c\cache-1dabca71be75cbef.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

,label,text
0,1,sports fans - get the latest sports news str* ...
1,0,It's justbeen overa week since we broke up and...
2,0,Not directly behind... Abt 4 rows behind ü...
3,0,"Haha, my legs and neck are killing me and my a..."
4,0,Me too baby! I promise to treat you well! I be...
...,...,...
4495,1,Free entry in 2 a wkly comp to win FA Cup fina...
4496,1,I am hot n horny and willing I live local to y...
4497,0,Me too watching surya movie only. . .after 6 p...
4498,0,"Good evening Sir, hope you are having a nice d..."


In [22]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [23]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore

from textattack.augmentation import Augmenter
from textattack.augmentation.recipes import EasyDataAugmenter





# Create augmenter with specified parameters
augmenter = EasyDataAugmenter(pct_words_to_swap=0.1, transformations_per_example=1)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):
    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)



time_end = time.perf_counter()
tim = time_end - time_sta

In [25]:


print(tim/3600)




0.4173859596666668


In [26]:
display(results_list)

[['sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE SPORT ON to www..co.uk 0870141701216+ norm 4txt/120p '],
 ["It's justbeen overa week since we broke up and already our brains are going to mush!"],
 ['Not directly behind... Abt 4 rows behind...'],
 ['Haha, my legs and neck are killing me and my amigos are hoping to end the night with a burn, think hope I could swing by in like an hour?'],
 ['Me too baby! I promise to treat you well! I iodine bet you will take good care of me...'],
 ['Enjoy ur Ur life. . Good night'],
 ['Sday only joined.so training we depart today:)'],
 ['Prof: you have passed in all the papers in this sem congrats . . . . Student: Enna kalaachutaarama..!! Prof:???? Gud mrng!'],
 ['Hey whats up? U sleeping all aurora?'],
 ['Have give your lunch and come quickly and open the door:)'],
 ['to stopped. Going back Cramps sleep'],
 ["MAKE SURE ALEX KNOWS HIS BIRTHDAY IS OVER amp IN FIFTEEN MINUTES AS FAR AS YOU'RE CONCERNED"],
 ['"GRAN O

In [27]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p '],
 ["It's justbeen overa week since we broke up and already our brains are going to mush!"],
 ['Not directly behind... Abt 4 rows behind ü...'],
 ['Haha, my legs and neck are killing me and my amigos are hoping to end the night with a burn, think I could swing by in like an hour?'],
 ['Me too baby! I promise to treat you well! I bet you will take good care of me...'],
 ['Enjoy ur life. . Good night'],
 ['Sday only joined.so training we started today:)'],
 ['Prof: you have passed in all the papers in this sem congrats . . . . Student: Enna kalaachutaarama..!! Prof:???? Gud mrng!'],
 ['Hey whats up? U sleeping all morning?'],
 ['Have your lunch and come quickly and open the door:)'],
 ['Cramps stopped. Going back to sleep'],
 ["MAKE SURE ALEX KNOWS HIS BIRTHDAY IS OVER IN FIFTEEN MINUTES AS FAR AS YOU'RE CONCERNED"],
 ['"GRAN ONLYFO

In [28]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,sports fans - get the latest sports news str* ...,sports fans - get the latest sports news str* ...
1,It's justbeen overa week since we broke up and...,It's justbeen overa week since we broke up and...
2,Not directly behind... Abt 4 rows behind ü...,Not directly behind... Abt 4 rows behind...
3,"Haha, my legs and neck are killing me and my a...","Haha, my legs and neck are killing me and my a..."
4,Me too baby! I promise to treat you well! I be...,Me too baby! I promise to treat you well! I io...
...,...,...
4495,Free entry in 2 a wkly comp to win FA Cup fina...,receive entry in 2 a wkly comp to win FA Cup f...
4496,I am hot n horny and willing I live local to y...,I am hot n horny and willing I live local to y...
4497,Me too watching surya movie only. . .after 6 p...,Maine too watching surya movie only. . .after ...
4498,"Good evening Sir, hope you are having a nice d...","Good evening Sir, hope you are having a nice d..."


In [29]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,sports fans - get the latest sports news str* ...,sports fans - get the latest sports news str* ...
1,Not directly behind... Abt 4 rows behind ü...,Not directly behind... Abt 4 rows behind...
2,"Haha, my legs and neck are killing me and my a...","Haha, my legs and neck are killing me and my a..."
3,Me too baby! I promise to treat you well! I be...,Me too baby! I promise to treat you well! I io...
4,Enjoy ur life. . Good night,Enjoy ur Ur life. . Good night
...,...,...
4423,Free entry in 2 a wkly comp to win FA Cup fina...,receive entry in 2 a wkly comp to win FA Cup f...
4424,I am hot n horny and willing I live local to y...,I am hot n horny and willing I live local to y...
4425,Me too watching surya movie only. . .after 6 p...,Maine too watching surya movie only. . .after ...
4426,"Good evening Sir, hope you are having a nice d...","Good evening Sir, hope you are having a nice d..."


In [30]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\augmented_datasets\EDA\run3\dataset.csv')